In [12]:
import requests
from bs4 import BeautifulSoup
import re
import time
import pandas as pd

In [2]:
categories = ['EDUCATION', 'BUSINESS', 
       'DATING', 'SPORTS', 
       'WEATHER', 'FOOD_AND_DRINK', 
       'HEALTH_AND_FITNESS', 'BEAUTY', 
       'MUSIC_AND_AUDIO', 'NEWS_AND_MAGAZINES', 
       'SOCIAL', 'SHOPPING', 
       'PRODUCTIVITY', 'PHOTOGRAPHY', 
       'MEDICAL', 'PARENTING', 
       'EDUCATION', 'BEAUTY'
       'COMMUNICATION', 'TOOLS']

In [3]:
def get_links(data):
    '''Given html data this will extract all the links'''
    link_list = []
    for link in data.findAll('a'):
        get_link = link.get('href')
        link_list.append(get_link)
    return link_list

In [4]:
sports = requests.get('https://play.google.com/store/apps/category/BUSINESS/collection/topselling_free')
sports_html = sports.text #data
sports_data = BeautifulSoup(sports_html)
links = get_links(sports_data)
del links[0:107]
sports_links = list(set(links))
new_list = [app for app in sports_links if app.find('details') != -1] 

In [5]:
len(new_list)

60

In [6]:
def google_scraper(categories):
    '''Will cycle through every category and then request the site, pass the soup to 
        get_links and then clean the list and append results to a new list in a dict
        format that has the format of {cat_name : [link list]}'''
    listy = []
    for category in categories:
        url = f'https://play.google.com/store/apps/category/{category}/collection/topselling_free'
        r = requests.get(url)
        html = r.text
        soup = BeautifulSoup(html)
        links = get_links(soup)
        del links[0:107]
        clean = list(set(links))
        new_list = [app for app in sports_links if app.find('details') != -1]
        listy.append({category : new_list })
        time.sleep(3)
    return listy


In [7]:
cat = categories

In [9]:
stuff = google_scraper(cat)

In [ ]:
len(stuff[0][cat[0]])

In [11]:
x = 0
for idx, c in enumerate(cat):
    x += len(stuff[idx][c])
    print(x)

60
120
180
240
300
360
420
480
540
600
660
720
780
840
900
960
1020
1080
1140


In [13]:
df = pd.DataFrame(stuff)

In [14]:
df

,BEAUTY,BEAUTYCOMMUNICATION,BUSINESS,DATING,EDUCATION,FOOD_AND_DRINK,HEALTH_AND_FITNESS,MEDICAL,MUSIC_AND_AUDIO,NEWS_AND_MAGAZINES,PARENTING,PHOTOGRAPHY,PRODUCTIVITY,SHOPPING,SOCIAL,SPORTS,TOOLS,WEATHER
0,NaN,NaN,NaN,NaN,[/store/apps/details?id=com.logmein.gotowebina...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,[/store/apps/details?id=com.logmein.gotowebina...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,[/store/apps/details?id=com.logmein.gotowebina...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[/store/apps/details?id=com.logmein.gotowebina...,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[/store/apps/details?id=com.logmein.gotowebina...
5,NaN,NaN,NaN,NaN,NaN,[/store/apps/details?id=com.logmein.gotowebina...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,[/store/apps/details?id=com.logmein.gotowebina...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,[/store/apps/details?id=com.logmein.gotowebina...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[/store/apps/details?id=com.logmein.gotowebina...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[/store/apps/details?id=com.logmein.gotowebina...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
